In [20]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from typing import List
from torch.utils.data import Dataset, DataLoader

In [21]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [59]:
import tiktoken

class Tokenizer:
    def __init__(self, chars, vocab_size) -> None:
        self.chars = chars
        self.vocab_size = vocab_size

    def encode(self, text: str) -> List[int]:
        stoi = { ch:i for i,ch in enumerate(self.chars) }
        return [stoi[c] for c in text] 

    def decode(self, integers: List[int]) -> str: 
        itos = { i:ch for i,ch in enumerate(self.chars) }
        return ''.join([itos[i] for i in integers]) 

class OpenAITokenizer:
    def __init__(self) -> None:
        self.enc = tiktoken.get_encoding("gpt2")
        self.vocab_size = self.enc.n_vocab

    def encode(self, text: str) -> List[int]:
        return self.enc.encode(text)

    def decode(self, lst: List[int]) -> str: 
        return ''.join([self.enc.decode(el) for el in lst])  
    
class ShakespareDataset(Dataset):
    def __init__(self, text, tokenizer, block_size):
        self.data = torch.tensor(tokenizer.encode(text), dtype=torch.long)
        self.block_size = block_size

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        x = self.data[idx:idx+self.block_size]
        y = self.data[idx+1:idx+self.block_size+1]
        x, y = x.to(device), y.to(device)
        return x, y

In [23]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [64]:
import wandb

# start a new wandb run to track this script
wandb.init(
    project="mini-GPT",
    
    # track hyperparameters and run metadata
    config={
        "learning_rate": learning_rate
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: r-alvarezlucendo16 (neural-object-tracking). Use `wandb login --relogin` to force relogin


In [26]:
import numpy as np

@torch.no_grad()
def estimate_loss(model, dataloader, eval_iters):
    losses = []
    model.eval()
    for k in range(eval_iters):
        X, Y = next(iter(dataloader))
        # compute loss
        logits, loss = model(X, Y)
        losses.append(loss.item())
    model.train()
    return np.mean(losses)

def train(train_loader, test_loader, model, optimizer, max_iters, eval_interval, eval_iters):
    for i in range(max_iters):
        if i % eval_interval == 0 or i == max_iters - 1:
            train_loss = estimate_loss(model, train_loader, eval_iters)
            val_loss = estimate_loss(model, test_loader, eval_iters)
            wandb.log({"train_loss": train_loss, "val_loss": val_loss})
            # print(f"step {i}: train loss {train_loss:.4f}, val loss {val_loss:.4f}")
    
        batch = next(iter(train_loader))
        x, y = batch
        logits, loss = model(x, y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

In [61]:
torch.manual_seed(1337)
 
# read file.
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# create tokenizer.
chars = sorted(list(set(text))) 
vocab_size = len(chars)

tokenizer = Tokenizer(chars, vocab_size)
# tokenizer = OpenAITokenizer()

# train and test splits
dataset = ShakespareDataset(text, tokenizer, block_size)
n = int(0.9*len(dataset)) # first 90% will be train, rest val
train_dataset, test_dataset = torch.utils.data.dataset.random_split(dataset, [n, len(dataset) - n])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# create model.
model = BigramLanguageModel(vocab_size=tokenizer.vocab_size)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

train(train_loader, test_loader, model, optimizer, max_iters, eval_interval, eval_iters)



0.209729 M parameters
step 0: train loss 4.3972, val loss 4.3954
step 100: train loss 2.6554, val loss 2.6292
step 200: train loss 2.5091, val loss 2.5210
step 300: train loss 2.4274, val loss 2.4533
step 400: train loss 2.3579, val loss 2.4052
step 500: train loss 2.3136, val loss 2.3506
step 600: train loss 2.2566, val loss 2.3178
step 700: train loss 2.2175, val loss 2.2510
step 800: train loss 2.1842, val loss 2.2112
step 900: train loss 2.1547, val loss 2.2061
step 999: train loss 2.1003, val loss 2.1657


In [28]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = m.generate(context, max_new_tokens=2000)[0].tolist()
print(tokenizer.decode(generated))



PETNO:
Iull to eve vides teern end, chiss flare you of theave?


SUCUNTUK:
A, shal irn st, hen lordeng for hondeed,
Thou thun son will foom, govy that trootcioug Eurd, ast depesk.

OLI:Cer'sp ap. I I ssufflores whoused for be hich; my sut leank?

INIO:
Shay:
Makht fat you?


NUNG IO:
Co dhe enop arsit's wice of know,
Tre thirthe his tnobe hou that sirest dod reare, asd chell youk.

LOUBEERDNE:
Whils Ry of for le.


Thorse here's whage tho is doug Qpe hereve tasis to ou dry our,
cwheraurt ill: noto hat the to beek bro,
For not pagse know sto the do his-m aess bad,
And you known qut nentr'd dithteang and yin ou that hoongerse; then Ho he you:
Whe wintnn the oug a tour uspof dive.

KIINO:
Soldl. Thit 'tome?

ZUSNIUK:
Rorstithfirs in wement, llay lop.

PORUCHALWO:
Fay frestirses bok errses to begiard
And to pe hason with shou mave if I avards
not aur inte on the me sand-des a sueipe.

YLUL$IETR:
I the co' sme woth yee thath couses
Aht greand, hars tute, hou swath shere.

AUCUCIVHARY med 

In [55]:
enc = tiktoken.get_encoding("cl100k_base")
tokenizer = Tokenizer(chars, vocab_size)
word = 'hello how are you'
res = [s for s in word]
# print(len(tokenizer.encode(res)))
print(enc.encode(word))

[15339, 1268, 527, 499]
